# Server

In [ ]:
# import socket

# HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
# PORT = 8808        # Port to listen on (non-privileged ports are > 1023)

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('Listening socket server')
#     conn, addr = s.accept()
#     with conn:
#         print('Connected by', addr)
#         while True:
#             data = conn.recv(1024)
#             if not data:
#                 break
#             print(data)
#             conn.sendall(data)

Listening socket server


In [ ]:
# import socket programming library 
import socket 
import json

# import thread module 
from _thread import *
import threading 

def read_recv_data(data):
    a = 1

def saveToCSV(j):
    a = 1


print_lock = threading.Lock() 

# thread fuction 
def threaded(c): 
    while True: 
        data_str = ''
        success_uuids = []
        try:
            # data received from client 
            data = c.recv(1024)
            print("recv: " + data.decode('utf-8'))

            data_str += data.decode('utf-8')
            for d in data_str.splitlines():
                print(d)
                try:
                    j = json.loads(d)
                    uuid = j['uuid']
                    success_uuids.append(uuid)
                except:
                    data_str = d
                    
            c.send(json.dumps({'success': success_uuids}).encode() + b'\n')
            success_uuids = []
            
            if not data: 
                print('Bye') 

                # lock released on exit 
                print_lock.release() 
                break

            # reverse the given string from client 
    #         data = data[::-1] 

            # send back reversed string to client 
            c.send(data+b'\n') 
        except ConnectionResetError:
            print("error")
            print_lock.release()
            break
            
    # connection closed 
    c.close() 
  
  
def Main(): 
    host = "127.0.0.1" 
#     host = "192.168.1.104"
        
    # reverse a port on your computer 
    port = 8808
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    s.bind((host, port)) 
    print("socket binded to post", port) 
  
    # put the socket into listening mode 
    s.listen(5) 
    print("socket is listening") 
  
    # a forever loop until client wants to exit 
    while True: 
  
        # establish connection with client 
        c, addr = s.accept() 
  
        # lock acquired by client 
        print_lock.acquire() 
        print('Connected to :', addr[0], ':', addr[1]) 
        c.send(b'{"test":"test"}\n')
  
        # Start a new thread and return its identifier 
        start_new_thread(threaded, (c,)) 
    s.close() 

Main()


socket binded to post 8808
socket is listening
Connected to : 127.0.0.1 : 43300
recv: {"action":1,"data":"STATE:1:cr8300c\/cr14200c\/cf:4c9d|4sTs\/3cAhAs\/3d:14200|-14200","type":1,"uuid":"03ce94ba-8ba5-442b-95f6-9f4892e0ea58"}

{"action":1,"data":"STATE:1:cr8300c\/cr14200c\/cf:4c9d|4sTs\/3cAhAs\/3d:14200|-14200","type":1,"uuid":"03ce94ba-8ba5-442b-95f6-9f4892e0ea58"}
recv: {"action":1,"data":"STATE:2:r4350c\/cc\/cr5550c\/cc:8h5s|8c4h\/AhKhTd\/4d\/Jd:-5550|5550","type":1,"uuid":"78543995-2471-46c1-8177-381be30149f6"}

{"action":1,"data":"STATE:2:r4350c\/cc\/cr5550c\/cc:8h5s|8c4h\/AhKhTd\/4d\/Jd:-5550|5550","type":1,"uuid":"78543995-2471-46c1-8177-381be30149f6"}
recv: {"action":1,"data":"STATE:3:cc\/cr10400c\/cc\/cr10800c:7c6d|8c3d\/JsJhAc\/8s\/6h:-10800|10800","type":1,"uuid":"3d1e02c4-81bb-456b-8faf-4a41d8542c2f"}

{"action":1,"data":"STATE:3:cc\/cr10400c\/cc\/cr10800c:7c6d|8c3d\/JsJhAc\/8s\/6h:-10800|10800","type":1,"uuid":"3d1e02c4-81bb-456b-8faf-4a41d8542c2f"}
recv: {"action":1,"da